<a href="https://colab.research.google.com/github/faykar-cyber/Telegram-auto-forward-/blob/master/my_telegram_automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install telethon

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.9/715.9 kB 16.2 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=9f66140beadee9a0408726b95c896e7ad83077bc903c9f17cda36537ac07ebeb
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [ ]:
import asyncio
import os
from telethon import TelegramClient
from telethon.sessions import SQLiteSession
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set session file path inside Google Drive
session_path = "/content/drive/MyDrive/telegram_session.session"

# Replace these with your API credentials
api_id = 24255892
api_hash = '0251d26f012dd2bd24e7a2e3135fca83'

# Use the session from Google Drive
session = SQLiteSession(session_path)
client = TelegramClient(session, api_id, api_hash)

async def login():
    try:
        print("Starting Telegram Client for login...")
        await client.start()
        print("Login successful. Session saved in Google Drive. You won’t need to log in again.")
    except Exception as e:
        print(f"Login failed: {e}")
    finally:
        await client.disconnect()

# Run login process
await login()

Mounted at /content/drive
Starting Telegram Client for login...
Login successful. Session saved in Google Drive. You won’t need to log in again.


In [1]:
!pip install telethon
import sys
import asyncio
from telethon import TelegramClient, events
from telethon.tl.types import InputPeerChannel, MessageMediaDocument, MessageMediaPhoto
from telethon.sessions import SQLiteSession

# Mount Google Drive
drive.mount('/content/drive')

# Add Telethon path from Google Drive
telethon_path = "/content/drive/MyDrive/telethon_lib"
sys.path.append(telethon_path)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Your Telegram API credentials
api_id_1 = '21097853'  # Replace with your API ID
api_hash_1 = '27f7c2e449ca2903765aa4dbed2c428'  # Replace with your API Hash

# Path to the session file stored in Google Drive
session_path = "/content/drive/MyDrive/telegram_session.session"
session = SQLiteSession(session_path)

# Initialize the Telegram client with persistent session
client = TelegramClient(session, api_id_1, api_hash_1)

# Replace these with actual IDs and access hashes
channels = [
    (2315854408, 3152840574677520582, 1881977634, 592818182236424971),
    (1958596918, 634969391175935554, 1881977634, 5928181822364249718),
    (1908991684, 6655715077810067354, 1881977634, 5928181822364249718),
    (2440920833, 3701624762256072702, 1881977634, 5928181822364249718),
    (2495856742, 3343034098660930539, 1881977634, 5928181822364249718),
    (2463792229, 7323715828442030437, 1881977634, 5928181822364249718),
    (2361184864, 4759118466964879818, 1881977634, 5928181822364249718),
    (1986865614, 9148856788000007725, 1881977634, 5928181822364249718),
    (2431170966, 8326569853277104679, 1881977634, 5928181822364249718),
    (1661102889, 4759118466964879818, 1881977634, 5928181822364249718),
    (2089653783, 3564760369603079005, 1881977634, 5928181822364249718),
    (2031237062, 6241702314817245303, 1881977634, 5928181822364249718),
    (1597273610, 9155378247437216505, 1881977634, 5928181822364249718),
    (2451514128, 8811471449435759157, 1881977634, 5928181822364249718),
    (2088743453, 1619763026630941486, 1881977634, 5928181822364249718),
    (2104364419, 916295117729830232, 1881977634, 5928181822364249718),
    (1727505028, 64747657680901694, 1881977634, 5928181822364249718)
]

async def handle_messages(source_channel_id, source_channel_access_hash, target_channel_id, target_channel_access_hash):
    logger.info(f"Listening to {source_channel_id}...")
    print(f"Listening to {source_channel_id}...")

    source_channel = InputPeerChannel(source_channel_id, source_channel_access_hash)
    target_channel = InputPeerChannel(target_channel_id, target_channel_access_hash)

    @client.on(events.NewMessage(chats=source_channel))
    async def handler(event):
        try:
            file_name = None

            if not event.message.media:
                logger.info(f"Skipped text-only message from {source_channel_id}")
                print(f"Skipped text-only message from {source_channel_id}")
                return

            if isinstance(event.message.media, MessageMediaPhoto):
                logger.info(f"Skipped photo message from {source_channel_id}")
                print(f"Skipped photo message from {source_channel_id}")
                return

            if isinstance(event.message.media, MessageMediaDocument):
                document = event.message.media.document
                if hasattr(document, 'attributes'):
                    for attr in document.attributes:
                        if hasattr(attr, 'file_name'):
                            file_name = attr.file_name.lower()
                if file_name and file_name.endswith(('.torrent', '.exe', '.srt','.mp3', '.vob', '.html', '.ifo', '.jpeg','.txt', '.bup', '.png', '.doi', '.vtt', '.url', '.bin', '.mhtml', '.jpg', '.bin', '.ini', '.msi', '.bat', '.sh')):
                    logger.info(f"Skipped unsupported file from {source_channel_id}: {file_name}")
                    print(f"Skipped unsupported file from {source_channel_id}: {file_name}")
                    return

            await client.send_file(target_channel, file=event.message.media, caption=event.message.text or "")
            logger.info(f"Sent media from {source_channel_id}: {file_name if file_name else 'media'}")
            print(f"Sent media from {source_channel_id} to {target_channel_id}: {file_name if file_name else 'media'}")

        except Exception as e:
            logger.error(f"Error processing message from {source_channel_id}: {e}")
            print(f"Error processing message from {source_channel_id}: {e}")

async def main():
    while True:
        try:
            print("Starting Telegram Client...")
            await client.start()
            print("Telegram client started successfully.")
            await asyncio.gather(*[handle_messages(*channel) for channel in channels])
            await client.run_until_disconnected()
        except Exception as e:
            print(f"Connection lost: {e}. Reconnecting in 5 seconds...")
            await asyncio.sleep(5)
        finally:
            await client.disconnect()

# Run in Jupyter cell
await main()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.9/715.9 kB 8.5 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=c842ca20e8098dc0062c6e4f31e3eee859616551261f692066a1561f61b82ac0
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


NameError: name 'drive' is not defined